---
title: "Modelling pile driveability"
subtitle: "What's actually happening to the pile and soil during installation?"
author: "Kevin Duffy"
bibliography: ../bibliography.bib
---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/triaduct/offshoregeotechnics/blob/main/notes/8-2_modelling-pile-driving.ipynb){target="_blank"} <nbsp>

Determining the optimum pile geometry is an iterative process, where the trade-off between the higher capacities achieved using larger piles is offset by the increased difficulty and risk associated with driving these piles to the desired penetration. 

An overview of the driveability analysis is given in @fig-driveability-procedure, particularly with regards to the inputs and outputs of the wave equation analysis. These components are explored a little further in this section.

![Driveability procedure](pics/installation/driveability-procedure.png){#fig-driveability-procedure width=80%}

## Pile hammer
We can use the principle of energy conservation to model the hammer impact. Once the ram falls, the energy gets transferred from potential to kinetic energy. This potential energy can be modelled as:

$$
E_{potential} = m_{ram}gh_e
$$

where $h_e$ is the effective (or equivalent) hammer stroke based on the additional acceleration provided by the hydraulic pistons. The maximum potential energy of the pile is also referred to as the **rated energy** of the hammer. 

If no energy losses were to occur in the system during hammer freefall, the kinetic energy at impact would be equal to the potential energy, giving a hammer velocity of $v_{impact} = \sqrt{2gh}$. 

However, losses invariably occur due to friction, ram misalignment or loss of pressure in the hydraulic system. These losses are quantified by the **hammer efficiency** $\eta$ whereby:

$$ 
\eta_{hammer} = \frac{E_{impact}}{E_{potential}}
$${#eq-hammer-efficiency}

Typical values of $\eta = 0.5–0.8$ are assumed for diesel hammers, whereas for hydraulic hammers $\eta = 0.95-1.0$.

## Energy transfer from hammer to pile
When the hammer strikes the pile, kinetic energy from the falling ram is transferred to the pile--hammer system, generating a series of stress waves which propagate down the pile. This energy transfer is expressed as a function of time $t$:

$$
E(t) = \int F(t)V(t)\,dt
$$

where $F$ is the force and $V$ is the velocity. 

An example of some force measurements at the pile head under one hammer blow is given in @fig-pda-reading. This figure shows a short period of five milliseconds where the ram is briefly contacts the pile head (via the anvil) before the rebounding back upwards. Reflection of waves back up the pile means that more compressive strains are also measured, as shown by the fluctuations after 25 milliseconds. 

The maximum energy generated in the pile during this time period is known as **EMX**. 

![Example of the force measured at opposing sides ($F_1$ and $F_2$) of a tubular pile [@buckleyOptimizationImpactPile2020]](pics/installation/buckley20-fig6.png){#fig-pda-reading width=60%}


:::{.callout-tip title="Measuring the transferred energy with a pile driving analyser (PDA)" collapse="true"}
A **pile driving analyser** measures the actual force imparted on the pile. It consists of two measurement devices that are affixed just underneath the pile head:

1. Accelerometer: for measuring the change in pile velocity.
2. Strain gauge: for measuring change in strain in the pile, which can then be converted to a normal force ($F = EA\varepsilon$).

Using these measurements, PDAs can then be used to both verify the driveability predictions, as well as being used in a signal matching analysis (e.g. through the software CAPWAP or IMPACT) to give an *approximate* estimate of the pile base and shaft capaciites. 

::: {#fig-pda layout-ncol=2}
![Sensors used in a PDA [(source)](https://www.g-octopus.com/equipment/pile-driving-analyzer-pda-system/)](pics/installation/pda1.jpg)

![PDAs being used offshore [(source)](https://allnamics.com/en/allnamics-offshore-services/)](pics/installation/pda2.jpg){width=60%}

Pile driving analyser
:::

:::
<!-- #End of Quarto callout -->



Just like we how we use $\eta$ to quantify the energy loss *within* the hammer itself (@eq-hammer-efficiency), there is also an energy loss when the hammer hits the anvil. This can be expressed as: 

$$
\alpha_{rated} = \frac{EMX}{E_{rated}}
$$

or as @flynnDrivenCastinsituPiles2019 suggested, as a function of the actual impact energy:

$$
\beta = \frac{EMX}{E_{impact}}
$$

The true energy transfer ratio falls below unity due to noise and heat generated at impact.


:::{.callout-tip title="Example: Maximum force applied on the pile head" collapse="true"}

By equating the potential energy to the kinetic energy, we can derive the initial velocity of the pile after the hammer blow as well as the force applied on the pile.

The calculator below gives an example of this, applying Timoshenko's one-dimensional beam theory whereby: 

$$ 
v_{max} = \sqrt{2gh}
$$
where $g$ is the gravitational acceleration (9.81 m/s<sup>2</sup>) and $h$ is the fall height of the ram. From here, the compressive stress applied can be obtained with:

$$ 
\sigma_{max} = \frac{v_{max}E}{c}
$$
where $E$ is the Young's Modulus and $c$ is the wave speed in the pile material (e.g. $c = 5125$ m/s in steel). 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/triaduct/offshoregeotechnics/blob/main/notes/8-2_modelling-pile-driving.ipynb){target="_blank"} <nbsp>

In [ ]:
import numpy as np 

# NB: The script behind the calculator is in JavaScript, although here is the Python equivalent
def force_imparted_by_hammer(Do, t, fall_height):
    """
    Simple example based on Timoshenko's 1D beam theory. 

    Do: Outer diameter [m]
    t:  Wall thickness [mm]
    fall_height: Fall height of the ram [-]
    
    Returns maximum impact force on the pile, assuming 100% efficiency
    """
    # Constants
    c_steel = 5125 # Wave speed in steel [m/s]
    E_steel = 210 # Youngs modulus GPa
    
    # Supporting calculations
    D_inner = D_outer - wall_thickness*1e-3*2   # Inner pile diameter [m]
    A_annulus = np.pi*(D_outer/2)**2 - np.pi*(D_inner/2)**2   # Area of the annulus [m2]

    # Main calculations
    v = np.sqrt(2*9.81*fall_height) # velocity [m/s]
    Fmax = (v*E_steel*1e6*A_annulus)/c_steel # Fmax [kN]
    
    print(f"Max. compressive force is {Fmax:.0f}kN")
    return Fmax  # [kN]


<div style="max-width: 320px; margin: 15px auto; font-family: sans-serif; border: 1px solid #ccc; padding: 15px; border-radius: 8px; background-color: #f9f9f9; font-size: 14px;">
  <h4 style="text-align: center; margin-bottom: 15px;">Maximum impact force on a tubular pile</h4>

  <label style="display: block; margin-bottom: 8px;">
    Outer diameter (m)
    <input type="number" id="D_outer" step="0.05" min="0.05" style="width: 100%; padding: 4px; margin-top: 3px;">
  </label>

  <label style="display: block; margin-bottom: 8px;">
    Wall thickness (mm)
    <input type="number" id="t" step="1" min="1" style="width: 100%; padding: 4px; margin-top: 3px;">
  </label>

  <label style="display: block; margin-bottom: 12px;">
    Fall height (m)
    <input type="number" id="fallHeight" step="0.1" min="0.1" style="width: 100%; padding: 4px; margin-top: 3px;">
  </label>

  <button id="runBtn" style="width: 100%; padding: 8px; font-size: 14px; border-radius: 5px; background-color: #007acc; color: white; border: none; cursor: pointer;">
    Run
  </button>

  <p style="margin-top: 12px; font-weight: bold;">
    Result: <span id="result">—</span>
  </p>
</div>

<script>
function forceImpartedByHammer(Do, t, fallHeight) {
    const cSteel = 5125;      // wave speed in steel [m/s]
    const E = 210e9;          // Young’s modulus [Pa]

    // inner pile diameter [m]
    const Di = Do - t * 1e-3 * 2;

    // cross-sectional area of annulus [m2]
    const A = Math.PI * Math.pow(Do / 2, 2) - Math.PI * Math.pow(Di / 2, 2);

    // velocity from fall height [m/s]
    const v = Math.sqrt(2 * 9.81 * fallHeight);

    // max impact force [kN]
    const Fmax = (v * E * A) / cSteel / 1000;

    return Fmax; 
}

document.getElementById("runBtn").addEventListener("click", function() {
    const Do = parseFloat(document.getElementById("D_outer").value);
    const t = parseFloat(document.getElementById("t").value);
    const fallHeight = parseFloat(document.getElementById("fallHeight").value);

    if (isNaN(Do) || isNaN(t) || isNaN(fallHeight)) {
        document.getElementById("result").textContent = "Please fill in all fields.";
        return;
    }

    const result = forceImpartedByHammer(Do, t, fallHeight);
    document.getElementById("result").textContent = result.toFixed(0) + " kN";
});
</script>

As a simple example, the PDA readings by Buckley et al. (@fig-pda-reading) are considered. The pile had an outer diameter of 508 mm with a wall thickness of 20.6 mm. The pile was then driven using a Junttan SHK100-3 4T hammer. 

Judging by the [specification sheet online](https://junttan.com/wp-content/uploads/2015/11/16284_SHK100-3_Datasheet.pdf), the hammer has a maximum drop height of 1.2m. Using this, we can figure out the maximum possible energy and force that the hammer can impart on the pile---**giving an answer of 6271 kN**. 

This answer is three times higher than the maximum force from the PDA results in @buckleyOptimizationImpactPile2020. Although this is unsurprising: PDA results represent the true force that has been transferred to the hammer, whereas the example we just did does not reflect the impact efficiency. Furthermore, @buckleyOptimizationImpactPile2020 does not specify what the actual drop height was for the blow provided, and it's likely that this was smaller than the absolute limit of the hammer. 

Nevertheless, the simple example can be used a preliminary check for assessing if enough force can be mobilised from the hammer in order to overcome the soil resistance acting in the opposite direction.

:::
<!-- #End of Quarto callout -->

## Wave propagation through pile
The hammer impact propagates a stress wave through the pile, travelling at a speed $c$ (@fig-wave-propagation). The pile also has a resistance to this wave propagation, referred to as the pile impedance $Z$: 

$$
Z = EA/c
$$

where $E$ is the Young's modulus and $c$ is the wave speed through the pile ($c = \sqrt{E/\rho}$).

Any time there's a change in impedance---like between the pile and the soil---waves are reflected in the opposite direction (@fig-wave-propagation). This mean there upward and downward waves travelling through the pile at the same time, rapidly losing energy as they do so. 

![Wave propagation down a pile](pics/installation/stress-wave-propagation.png){#fig-wave-propagation}

To model this energy transfer, Newton's Law can be used, containing both a parameter for mass and acceleration:

$$
\rho \left(\frac{\delta^2 w}{\delta t^2}\right) = E \left(\frac{\delta^2 w}{\delta z^2}\right) \tag{15}
$$

which can be rewritten as:

$$
\left(\frac{\delta^2 w}{\delta t^2}\right) = c^2 \left(\frac{\delta^2 w}{\delta z^2}\right) \tag{16}
$$

where: $w$ is the displacement, $z$ is the downward coordinate, $t$ is time, $\rho$ is the mass density.

There are a couple of ways to solve these equations ([@fig-wave-equation-models]):

1. Lumped mass model (GRLWEAP)
2. Method of characteristics (AllWave, formerly known as TNOWAVE)

The lumped mass model simulates the pile as a series of masses and springs. The main disadvantage of the lumped mass model (compared to the method of characteristics) is that it's an approximate method as the force is calculated at the spring location whereas the velocity is at the mass location (half a segment length difference).

By contrast, the method of characteristics treats the pile as a continuous medium through which the stress wave can propagate. In this approach, the pile is divided into segments of uniform size, with the soil resistance represented as a concentrated load acting between the segments. The stress wave is assumed to travel unchanged through each individual segment and its amplitude is modified only at the junctions where soil resistance is mobilised or where changes in pile section occur. 

A key advantage of the method of characteristics is that both force and velocity are calculated at the same location in the pile, which makes the method particularly well-suited for wave matching. It also produces twice as many data points along the pile compared to the Smith lumped-mass model for the same time interval, leading to a finer temporal resolution of the pile response.

However, the limitation of the method is that the length of the elements must be chosen as integer multiples of the product of the time step and the wave propagation velocity. This restriction makes the method less convenient for simulating hammer and cushion behavior, since those components are better represented by springs and dashpots rather than fictitious material segments constrained by wave speed requirements. For these reasons, while the MoC provides a mathematically elegant description of wave propagation in piles, practical applications often prefer lumped-mass wave equation models for handling hammer–cushion–soil interactions.

![Types of wave equation models [@middendorp30YearsExperience2012]](pics/installation/wave-equation-models.png){#fig-wave-equation-models}

:::{.callout-note}
Commerical software is typically used for pile driveability analyses. The most popular software are:

1. GRLWEAP, based on the lumped mass model
2. AllWave, based on the method of characteristics
:::


## Lumped mass model 
To delve a little deeper into the inner workings of a wave equation analysis, the lumped mass model is taken as an illustrative example.

With this method, the pile is modelled as a discretised set of lumped masses, connected by springs. Each pile segment has a weight ($w_1$ to $w_n$) and are connected by springs ($k_1$ until $k_n−1$). Time is also discretised into smaller time intervals and the action of each element and spring is calculated for each and every time interval.

![Lumped mass model [@smithPiledrivingAnalysisWave1960]](pics/installation/lumped-mass.png){#fig-lumped-mass-model}

### Pile-soil interaction
To "complete" our analytical model, we need to define how the pile interacts with the soil. This is often done using what's called a **spring-dashpot** (or spring-damper) model. 

The core of most driveability assessments and software is based on the rheological framework proposed by @smithPiledrivingAnalysisWave1960. @smithPiledrivingAnalysisWave1960 proposed that the pile displacement $w$ and velocity $v$ as a function of a spring, dashpot and plastic slider (@fig-spring-dashpot) where the total resistance is given by:

$$ 
R_t = R_s + R_d
$$

where $R_s$ is the static resistance and $R_d$ is the dynamic (or damping) resistance. 

![Simulation of soil resistance in both the lumped mass model and method of characteristics](pics/installation/spring-dashpot.png){#fig-spring-dashpot}

:::{#fig-spring-dashpot-video}
<iframe width="560" height="315" src="https://www.youtube.com/embed/vLaFAKnaRJU?si=oHztbbyP3tPm0Jmg&amp;start=366" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

Visualisation of the spring-dashpot model
:::

### Static resistance $R_s$
The static resistance is a function of the displacement of the pile with respect to the soil, and is therefore present during both static and dynamic loading. 

To define the spring model, two main inputs are provided (@fig-spring-model): the maximum static resistance given by a static resistance to driving (SRD) model and the quake, which is the displacement are which the spring exhibits plastic behaviour.

![Spring model for determining the static resistance](pics/installation/spring.png){#fig-spring-model width=80%}

#### Static resistance to driving (SRD) model
To define the maximum static resistance $R_{s,max}$, a **static resistance to driving model** is used. An SRD model gives a profile of the shaft and base resistance developed during pile installation. While they are quite similar to the axial capacity methods we looked at earlier, there are some key differences:

* Static capacity methods include some degree of aging, whereas SRD methods give the instanteous capacity at the end of driving
* Influence of the soil plug is very different under dynamic loading compared to static loading because of the plug's intertia
* SRD methods generally use best or upper-bound estimates of resistance, instead of lower bound estimates. This is to ensure that the chosen hammer can be used for all piles at a given site, reducing the likelihood of having to return onshore to get another hammer
* Pore pressure development under high strains

There are also many different SRD methods recommended in public literature (e.g. @toolanGeotechnicalPlanningPiled1977, @stevensEvaluatingDrivabilityHard1982, @almSoilModelPile2001, @jonesCPTbasedSoilResistance2020), including recent research by @lehaneApplicationUnifiedCPT2022 into the application of the Unified pile design method for SRD models. 

These methods are generally calibrated based on pile test database, so the efficacy of each method is highly dependent on the database upon which its calibrated, as well as the selection of inputs to the driveability model.

#### Quake
The soil quake is the maximum elastic deformation after which further displacement causes slippage between pile and soil. It also encapsulates the rebound of the pile after being subjected to the hammer push.

Shaft quakes vary relatively little, with a quake of 2.5 mm generally considered reasonable for most soils. However, toe quakes can vary widely, as the value is highly dependent on the soil stiffness around the pile base—which is in turn dependent on the installation procedure and the soil type.


### Dynamic resistance $R_d$
Pile driving is a dynamic process, and so energy itself is transferred from pile-to-soil which is turned dampened (i.e. energy is lost) by molecular interactions in the soil itself. The dynamic resistance is modelled as a linear function of the velocity of the pile, whereby:

$$
R_d =  R_s J_s v
$${#eq-smith-damping-factor}

where $J_s$ is the Smith damping factor [s/m]. This essentially changes the static resistance profile to something similar to @fig-smith-soil-model.

Although the Smith model was developed over half a century ago, the same basic factors for damping along the shaft, i.e., 0.15 s/m for sands and 0.65 s/m for clays are still recommended. Different values are used where measurements are made or in mixed soils. Only the base damping recommendation changed to 0.5 s/m for all soil types

![Influence of adding in the dynamic component to the total resistance [@smithPiledrivingAnalysisWave1960]](pics/installation/smith-soil-model.png){#fig-smith-soil-model}

:::{.callout-note}
@eq-smith-damping-factor is just one way of quantifying the dynamic resistance, although this is arguably an overly empirical method. Therefore, several other formulations also exist and are implemented in commercial software, such as Case Damping, Coyle-Gibson Damping or Rausche Damping.

:::

## Signal matching 
Pile driveability analyses focus on efficiency, since they act as a predictive tool that is used before installation. The primary outputs from this model are the simulated driving stresses, blow counts, load profiles and energy transfer.

However, signal matching software focus on a more detailed assessment of wave propagation by comparing these predictions to the measurements from a PDA. Using the method of characteristics, smaller time intervals are considered in the analysis and generally focussing on data collected from just a small number of blows. Through this comparison and analysis of the inputs to the wave equation, signal matching allows for a rough estimate of the pile base and shaft capacities. 

Therefore, signal matching software (e.g. CAPWAP, IMPACT) are primarily *interpretative* tools whereas driveability software (GRLWEAP, AllWave) are *predictive* tools.